In [ ]:
accelX=[]
times=[]

In [8]:
from flask import Flask,render_template,jsonify
from flask_sockets import Sockets
import matplotlib.pyplot as plt

#from scipy.signal import savgol_filter
#from scipy.interpolate import spline
from scipy.signal import lfilter
import slidingwindow as sw
import pandas as pd
import statistics
import numpy as np
import time
import keras
from keras.models import Sequential
from keras.layers import Dense
from scipy.spatial.distance import cosine
from scipy.sparse import csr_matrix
from keras.utils import to_categorical


from matplotlib import animation
from drawnow import drawnow, figure

%matplotlib inline
from IPython.display import display, clear_output



from keras.models import load_model
#LLLLLLLLLLLLLOOOOOOOOOOOOKKKKKKKKKKKKKKK HEEEEEERRRRRRRRRREEEEEEEEEEEEE




# fig = plt.figure()
# plt.rc('font', size=8)

# for i in range(6):
#     ax = fig.add_subplot(7, 1, (i+2))
#     ax.set_xlim([0, 100])
#     if i < 3:
#         ax.set_ylim([-2, 2])
#     else:
#         ax.set_ylim([-100, 100])
#     l, = ax.plot([])
#     lines.append(l)

# classifyLabel = plt.text(30, 1500, '--', style='italic', fontsize=30, bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})

# plt.ion()
# plt.show()





def update_line(lines, classifyLabel):
    running_acc_data=pd.DataFrame(accelX)
    running_gyro_data=pd.DataFrame(gyroX)
    currentClassifyLabel=classifyLabel
    minSize = min(running_acc_data.shape[0], running_gyro_data.shape[0])
    imu_data = np.append(running_acc_data[:minSize], running_gyro_data[:minSize], axis=1)

    
    for i in range(2):
        d1 = np.transpose(imu_data[:, i])
        dd = np.append(np.expand_dims(np.array(range(imu_data.shape[0])), axis=0), np.expand_dims(d1, axis=0), axis=0)
        lines[i]=dd
    return lines[0], lines[1], lines[2], lines[3], lines[4], lines[5],






activities=["Nothing","nailbiting","smoke","eat","face touch","still","IDK"]
window_size=61
model = load_model('Theeeeeeeeemodel.h5')


mylist = ["Ax", "Ay", "Az","Gx", "Gy", "Gz", "Mx", "My","Mz"]
# Returns a compiled model identical to the previous one

n_training = 30  # number of trials used for training, the rest for validation
n_components = 40 # PCA components
imu_dimensions = 4 # number of dimensions in imu data

classes = ['move', 'scratch', 'work', 'relax']




def thetruth(arr):
    for i in range(len(arr[0])):
        if arr[0,i]==True:
            print(i)
            return activities[i]
    return activities[6]

def timeSeriesToSupervised(df,var_list,window_size=1, dropnan=True):
    for var in var_list:
        for idx in range(window_size,0,-1):
            var_name = var + '(t+' + str(idx) + ')'    
            df[var_name] = df[var].shift(-idx)

    if dropnan:
        df.dropna(inplace=True)
    return df


starttime = time.time()

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1) 



app = Flask(__name__)

sockets = Sockets(app)
lines = [0,0,0,0,0,0,0]
accelX=[]
accelY=[]
accelZ=[]
times=[]
gyroX=[]
gyroY=[]
gyroZ=[]
magX=[]
sadtimes=[]
magY=[]
magZ=[]
rangeval = 20



global accelX
    

def converter(types, val):
    a = []
    temp = ""
    for x in val:
        if x == ",":
            a.append(float(temp))
            temp = ""
        else:
            temp += x
    a.append(float(temp))
    thisdict = {}
    if types in ("accel", "gyro", "mag"):
        thisdict = {"x": a[0], "y": a[1], "z": a[2]}
    elif types == "oriet":
        thisdict = {"azmuth": a[0], "pitch": a[1], "roll": a[2]}

    return thisdict


def Filter(arr):
    M=mean(arr)
    sd=stdev(arr)
    th=sd*2
    #arr(map(lambda x: ))
    return arr


@sockets.route('/accelerometer')
def echo_socket(ws):
    sadtimes=[]
    count=0
    f = open("accelerometer.txt", "a")
    while True:
        message = ws.receive()
        a = converter("accel", message)
        accelX.append(a['x'])
        accelY.append(a['y'])
        accelZ.append(a['z'])
        ts=time.time()
        times.append(ts)
        sadtimes.append(ts - starttime)
        sadtimes = list(filter(lambda x: x >= max(0, max(times) - rangeval), times))
        sadaccelX = accelX[max(len(times) - len(sadtimes), 0): ]
        sadaccelY = accelY[max(len(times) - len(sadtimes), 0): ]
        sadaccelZ = accelZ[max(len(times) - len(sadtimes), 0): ]
        count+=1
        #print('ooooooooooooooooooooooooooooo')
        if len(accelX)>= window_size and len(accelY)>= window_size and len(accelZ)>= window_size and len(gyroX)>= window_size and len(gyroY)>= window_size and len(gyroZ)>= window_size and len(magX)>= window_size and len(magY)>= window_size and len(magZ)>= window_size and count>=window_size:
            
            df1=np.array(accelX)
            df2=np.array(accelY)
            df3=np.array(accelZ)
            df4=np.array(gyroX)
            df5=np.array(gyroY)
            df6=np.array(gyroZ)
            df7=np.array(magX)
            df8=np.array(magY)
            df9=np.array(magZ)
            df0=np.array(times)
            
            acc_data1 = df1[-1*window_size:]
            acc_data2 = df2[-1*window_size:]
            acc_data3 = df3[-1*window_size:]
            gyr_data1 = df4[-1*window_size:]
            gyr_data2 = df5[-1*window_size:]
            gyr_data3 = df6[-1*window_size:]
            mag_data1 = df7[-1*window_size:]
            mag_data2 = df8[-1*window_size:]
            mag_data3 = df9[-1*window_size:]
            timer=df0[-1*window_size:]
            
            o=timer.reshape(-1,1)
            a=acc_data1.reshape(-1,1)
            b=acc_data2.reshape(-1,1)
            c=acc_data3.reshape(-1,1)
            d=gyr_data1.reshape(-1,1)
            e=gyr_data2.reshape(-1,1)
            f=gyr_data3.reshape(-1,1)
            g=mag_data1.reshape(-1,1)
            h=mag_data2.reshape(-1,1)
            i=mag_data3.reshape(-1,1)
            
         
            df=np.concatenate([o,a,b,c,d,e,f,g,h,i], axis=1)
            print('e')
            
            dataset = pd.DataFrame({'Time_Stamp':df[:,0],'Ax':df[:,1],'Ay':df[:,2],'Az':df[:,3],'Gx':df[:,4],'Gy':df[:,5],'Gz':df[:,6],'Mx':df[:,7],'My':df[:,8],'Mz':df[:,9]})
            #dataset.to_excel("output.xlsx")
            dft= timeSeriesToSupervised(dataset,mylist,window_size-1)
            x = dft.drop('Time_Stamp', axis=1)
            y_pred = model.predict(x)
            print(y_pred)
            #*******************************************************************************
            y_pred = (y_pred > 0.4)
            #*******************************************************************************
            count=window_size/2
            print(y_pred)
            vara=thetruth(y_pred)
            print(vara)
            update_line(lines, vara)
#             dataset.iloc[0:0]
#             accelX.clear()
#             accelY.clear()
#             accelZ.clear()
#             times.clear()
#             gyroX.clear()
#             gyroY.clear()
#             gyroZ.clear()
#             magX.clear()
#             magY.clear()
#             magZ.clear()
            #yhat = savgol_filter(accelX,10, 3)
            #b=np.asarray(accelX)
#             n = 11  # the larger n is, the smoother curve will be
#             b = [1.0 / n] * n
#             a = 1
#             yhat = lfilter(b,a,accelX)
            #yhat=np.linspace(b.min(),b.max(),11)
            #df = pd.DataFrame({'col':sadaccelX})
            
            ax.set_xlim(0, max(max(sadtimes), 60))
            ax.cla()
            ax.plot(sadtimes, sadaccelX)
            ax.plot(sadtimes,sadaccelY, color='red')
            ax.plot(sadtimes,sadaccelZ, color='yellow')
            #ax.plot(sadtimes,df.col.rolling(9).mean(),color='grey')
            display(fig)

            clear_output(wait = True)
            plt.pause(0.005)
            

            
            
#         plt.scatter(ts, a['x'])
        
#         plt.show()
#         #plt.clf()
#         plt.plot( times, accelX)
#         plt.show()
#         plt.pause(0.0001)

        #ax1.plot( times, accelY)
        #ax1.plot( times, accelZ)
        #print("hello");
        #ax.plot(a["x"], a["y"])
        #fig.show()
        #plot_cont(a["x"], 60)
        ws.send(message)
        
    


@sockets.route('/gyroscope')
def echo_socket(ws):
    f = open("gyroscope.txt", "a")
    while True:
        message = ws.receive()
        a = converter("gyro", message)
        gyroX.append(a['x'])
        gyroY.append(a['y'])
        gyroZ.append(a['z'])
        ws.send(message)
        #print(message, file=f)
    f.close()


@sockets.route('/magnetometer')
def echo_socket(ws):
    f = open("magnetometer.txt", "a")
    while True:
        message = ws.receive()
        a = converter("mag", message)
        magX.append(a['x'])
        magY.append(a['y'])
        magZ.append(a['z'])
       #print(message)
        ws.send(message)
       # print(message, file=f)
    f.close()


@sockets.route('/orientation')
def echo_socket(ws):
    f = open("orientation.txt", "a")
    while True:
        message = ws.receive()
        print(message)
        ws.send(message)
        print(message, file=f)
    f.close()


@app.route('/')
def hello():
    return render_template('charts.html')


@app.route('/data')
def helloo():
    return jsonify({'results' : accelX})




if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    sadtimes=[]
    server = pywsgi.WSGIServer(
        ('0.0.0.0', 5000), app, handler_class=WebSocketHandler)
    server.serve_forever()
    app.run()
    
    

    


Traceback (most recent call last):
  File "c:\python\python36\lib\site-packages\gevent\pywsgi.py", line 936, in handle_one_response
    self.run_application()
  File "c:\python\python36\lib\site-packages\geventwebsocket\handler.py", line 75, in run_application
    self.run_websocket()
  File "c:\python\python36\lib\site-packages\geventwebsocket\handler.py", line 52, in run_websocket
    list(self.application(self.environ, lambda s, h, e=None: []))
  File "c:\python\python36\lib\site-packages\flask\app.py", line 1997, in __call__
    return self.wsgi_app(environ, start_response)
  File "c:\python\python36\lib\site-packages\flask_sockets.py", line 45, in __call__
    handler(environment, **values)
  File "<ipython-input-8-d2455e59e18b>", line 181, in echo_socket
    sadtimes = list(filter(lambda x: x >= max(0, max(times) - rangeval), times))
  File "<ipython-input-8-d2455e59e18b>", line 181, in <lambda>
    sadtimes = list(filter(lambda x: x >= max(0, max(times) - rangeval), times))
Keyb

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import  numpy as np
accelX=[[1],[2],[3],[4],[5]]
accelY=[[6],[7],[8],[9],[10]]
accelZ=[[11],[12],[13],[14],[15]]

df1=np.array(accelX)
df2=np.array(accelY)
df3=np.array(accelZ)

window_size=5

acc_data1 = df1[-1*window_size:]
acc_data2 = df2[-1*window_size:]
acc_data3 = df3[-1*window_size:]

df=np.concatenate((acc_data1,acc_data2,acc_data3), axis=1)

dataset = pd.DataFrame({'ax':df[:,0],'ay':df[:,1],'az':df[:,2]})


dataset.ax.rolling(2).mean()

In [ ]:
# accelX
# df1=np.array(accelX)
# df1
# df2=np.array(accelY)
# df3=np.array(accelZ)
# acc_data1 = df1[-1*window_size:]
# acc_data2 = df2[-1*window_size:]
# acc_data3 = df3[-1*window_size:]
# a=acc_data3.reshape(-1,1)
# b=acc_data1.reshape(-1,1)
# c=acc_data2.reshape(-1,1)
# a.shape
# df=np.concatenate([b,c,a], axis=1)
# df



In [ ]:
d = {'T':times,'Ax':accelX,'Ay':accelY,'Az':accelZ}
Cdata=pd.DataFrame(d)
computeIMUFeatures(Cdata)


In [ ]:
#MOVING AVERAGE IS ALSO A GOOD FILTER FOR FOOT TAPPING YOLO
fig2 = plt.figure(figsize=(12,8))
axl = fig2.add_subplot(1, 1, 1) 

df = pd.DataFrame({'col':accelX})
#axl.plot(times,accelX)
#ax.plot(times,accelY, color='red')
#ax.plot(times,accelZ, color='yellow')
axl.plot(times,df.col.rolling(9).mean(),color='grey')
display(fig2)


In [ ]:

windows = sw.generate(np.array(accelX), sw.DimOrder.HeightWidthChannel, window_size, 0.5)



In [ ]:
print(f"From:{max(0, max(times) - 60)}, To:{max(max(times), 60)}")

In [ ]:
import pandas as pd

df=pd.DataFrame({'col':{1,2,3,4,5}})
df1=pd.DataFrame({'col':{11,21,13,14,15}})
df2=pd.DataFrame({'col':{111,211,113,114,115}})

d=(df('col'),df1('col'),df2('col'))
d

In [ ]:
# butterworths filter to noisy foot data seems usefull but not on naibiting

import pandas as pd  
import numpy as np
import matplotlib.pyplot 
from pandas import read_csv
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import signal

print(accelX)
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(1,1,1)
#ax1.plot(times,accelX)
fs = 1000
fc = 30  # Cut-off frequency of the filter
w = fc / (fs / 2) # Normalize the frequency
b, a = signal.butter(15, w, 'low')
output = signal.filtfilt(b, a, accelX)
t = times
# plt.plot(t, output, label='filtered')
ax1.plot(times,accelX, color='red')
ax1.plot(t,output, color='yellow')
display(fig)
